# k-Nearest Neighbors

Otra forma de clasificar elementos es a través del algoritmo de k-Nearest Neighbors. Este funciona clasificando los datos basándose a las etiquetas de los datos que tiene más cercanos en el espacio de características – para cada una nueva muestra se buscan los “k” vecinos más cercanos y dependiendo de estas etiquetas se decide a que clase pertenece dada nuevo elemento.

La forma de utilizarlo en scikit-learn es simple como cualquier otro clasificador, lo importamos del módulo:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Y tiene los métodos <code>fit</code> y <code>predict</code> usuales:

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=1000, random_state=42, noise=0.1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)

print(knn.predict(X_test))
print(knn.score(X_test, y_test))

Y también tiene el método <code>predict_proba</code> aunque la probabilidad aquí nos ayuda a definir cuantos vecinos cercanos tenía: 

In [ ]:
print(knn.predict_proba(X_test)[:20])

Como sea, lo interesante está en los argumentos, los hiperparámetros de la clase.

## Argumentos

Al igual que muchos otros modelos de machine learning, la clase <code>KNeighborsClassifier</code> tiene algunos argumentos para modificar su comportamiento:

 - <code>n_neighbors</code>: Este hiperparámetro determina la cantidad de vecinos que se utilizarán en la clasificación. Si el valor de <code>n_neighbors</code> es demasiado bajo, el modelo puede sobreajustar los datos, mientras que si el valor es demasiado alto, el modelo puede subajustar los datos. El valor por default 5.

 - <code>weights</code>: Este hiperparámetro determina cómo se ponderan las distancias entre las muestras de entrenamiento y la muestra de prueba. Las opciones son 'uniform', donde todas las muestras tienen el mismo peso en la clasificación, y 'distance', donde las muestras más cercanas tienen un mayor peso. Usualmente, la opción por defecto es 'uniform'.

 - <code>metric</code>: Este hiperparámetro determina la métrica de distancia utilizada para calcular las distancias entre las muestras. Algunas opciones comunes son 'euclidean', 'manhattan' y 'minkowski'.

 - <code>algorithm</code>: Este hiperparámetro determina el algoritmo utilizado para encontrar los vecinos más cercanos. Las opciones son 'brute', que busca los vecinos más cercanos calculando todas las distancias entre todas las muestras, y 'kd_tree' o 'ball_tree', que utilizan estructuras de datos para buscar los vecinos más cercanos de manera más eficiente.

Vamos a ver el comportamiento del modelo cuando modificamos su , comenzando por la que tal vez sea la más importante:

Primero vamos a crear un dataset de ejemplo:

In [ ]:
from utils import plot_boundaries

X, y = make_moons(n_samples=1000, random_state=42, noise=0.15)

## <code>n_neighbors</code>

In [ ]:
plot_boundaries(
    X, y, 
    [
        ('n_neighbors = 1', KNeighborsClassifier(n_neighbors=1)),
        ('n_neighbors = 10', KNeighborsClassifier(n_neighbors=10)),
        ('n_neighbors = 100', KNeighborsClassifier(n_neighbors=100)),
        ('n_neighbors = 999', KNeighborsClassifier(n_neighbors=999)),
    ]
)

## La importancia de escalar las características

k-Nearest Neighbors es un algoritmo basado completamente en las distancias entre características, es de vital importancia que estas estén escaladas antes de pasarlas al modelo, si no, vas a sufrir de problemas al momento de entrenar y obtener predicciones.

Para demostrarlo, aquí estoy creando un dataset y estoy sacando una de sus características fuera de la escala al multiplicarla por 5:

In [ ]:
X, y = make_moons(n_samples=100, random_state=42, noise=.1)
X[:,1] = X[:,1] * 5

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Después entreno un dataset con los datos sin escalar:

In [ ]:
knn_unscaled = KNeighborsClassifier(n_neighbors=5)
knn_unscaled.fit(X_train, y_train)
accuracy_unscaled = knn_unscaled.score(X_test, y_test)

Y entreno uno escalando las características previamente:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn_scaled = KNeighborsClassifier(n_neighbors=5)
knn_scaled.fit(X_train_scaled, y_train)
accuracy_scaled = knn_scaled.score(X_test_scaled, y_test)

De entrada podemos ver la diferencia entre el desempeño de uno y otro:

In [ ]:
print(f"Sin escalar\t{accuracy_unscaled:.4f}")
print(f"Escaladas\t{accuracy_scaled:.4f}")

Pero se ve puede apreciar mejor con una gráfica en dos dimensiones:

In [ ]:
from utils import plot_knn_boundaries

plot_knn_boundaries(knn_unscaled,knn_scaled, X_train, X_train_scaled, y_train)

## Tamaño

El tamaño de un modelo de kNN en disco y en memoria varía con respecto al tamaño de su dataset de entrenamiento:

In [ ]:
import joblib
import os
from sklearn.datasets import make_classification

n_samples = [100, 1000, 10000, 100000]

for n in n_samples:
    X, y = make_classification(n_samples=n, n_features=20)
    knn = KNeighborsClassifier()
    knn.fit(X, y)
    joblib.dump(knn, f"/tmp/knn_model_{n}.joblib")
    model_size = os.path.getsize(f"/tmp/knn_model_{n}.joblib")

    print(f"Tamaño del modelo (n={n}):\t{model_size:>10} bytes")

## En conclusión

El modelo de k-NN es uno que puedes utilizar en problemas de clasificación, especialmente en conjuntos de datos pequeños o de tamaño moderado, problemas con múltiples clases, datos ruidosos o valores faltantes, y en problemas con una dimensionalidad baja a moderada.

Pero considera no utilizarlo en problemas con conjuntos de datos grandes, problemas con una dimensionalidad alta, problemas en los que la velocidad es crítica, problemas con datos muy dispersos, y en problemas en los que la precisión es más importante que la simplicidad y la interpretabilidad. En estos casos, puede ser necesario considerar otros algoritmos de aprendizaje automático más adecuados para el problema específico.
